author:  Dr Milto Miltiadou
version: 1.0
created Oct 2024

In [1]:
import sys

# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'ee'
if modulename not in sys.modules: 
   # import GEE and Authenticate, token or log in will be asked from web browser
   import ee
   ee.Authenticate()
   ee.Initialize()
else:
   print('GEE already imported')
   # google earth engine already imported and authenticated

import pandas as pd
import numbers
from datetime import date
import os


In [ ]:
class Polygons:
    
    def __init__(self,shpfilename,polKeyCol):
        self.shp = shpfilename
        self.bufferredPoints = None
        self.df = None
        self.noOfShpPolygons = None
        self.shpPolygons = None
        self.polygonsList = None
        self.sampleSize = 400
           
        self.shpPolygons = ee.FeatureCollection(self.shp)
        self.noOfShpPolygons = self.shpPolygons.size().getInfo()
        self.polygonsList = self.shpPolygons.toList(self.noOfShpPolygons)
        print ("Shapefile ", self.shp, " is loaded with ", self.noOfShpPolygons, " polygons")
        self.currentSubset = None
        self.polygonID = polKeyCol
        self.exportStdVal        = True
        self.exportMeanVal       = True
        
    def getPolygons(self):
        return self.shpPolygons
        
        
    ## @brief method that returns the number of polygons
    def getLen(self):
       return self.noOfShpPolygons
   
    ## @brief method that sets a new sampling number (how many polygons will be processed in each iteration)
    #  @param[in] newSampling the new samplig size
    def setSampling(self,newSampling):
        self.sampleSize = newSampling
        print("Sampling size updated to", newSampling)
        
    
    # There is a copy of this function in Polygons 
    # Update both functions if something chang
    def createPolygons(self,currentMin,currentMax):
        self.currentSubset = None
        tmpList =  self.polygonsList.slice(currentMin,min(currentMax,self.noOfShpPolygons))
        self.currentSubset=ee.FeatureCollection(tmpList)
        return self.currentSubset
    
    ## @brief method that returns the number of polygons
    def getLenCurrentSub(self):
        return self.currentSubset.size().getInfo()
   
    # Copied from FieldData - 
    def getFeatureCollection(self,collection):
        bandNamesImg = collection.bandNames().getInfo()
        print('Band names: ', bandNamesImg)
        for band in bandNamesImg :
            if(not isinstance(band,str)):
                bandNamesImg.remove(band)
        featureCollection1 = ee.FeatureCollection([])
        for band in bandNamesImg:
            features = self.getInfoRegions(collection,band)
            featureCollection1 = featureCollection1.merge(features)        
        return featureCollection1
    
    # Modified from FieldData get mean and std for one band of an image for each buffered point
    def getInfoRegions(self,collection,bandName):
    # bnamestr = bandName.get('band')
        return collection.select([bandName]).reduceRegions(**{
            'collection': self.currentSubset.select(self.polygonID),
            'reducer': ee.Reducer.mean().combine(**{
                'reducer2': ee.Reducer.stdDev(),
                'sharedInputs': True
            }),
            'scale': 10#,
            #'bestEffort': True  # Use maxPixels if you care about scale.
        }).map(lambda feature: feature.set('bandName',bandName))  \
        .filter(ee.Filter.neq('mean',None))
    
    def processMatchesMean(self,row):
        # Get the list of all features with a unique row ID.
        matches = ee.List(row.get('matches'))
        # Map a function over the list of rows to return a list of column ID and value.
        values = matches.map(lambda feature: [ee.Feature(feature).get('bandName'), ee.Feature(feature).get('mean')])
        # Return the row with its ID property and properties for all matching column IDs storing the output of the reducer.
        return row.select([self.polygonID]).set(ee.Dictionary(values.flatten()))

    ## Fielddata: Format a table of triplets into a 2D table of rowId x colId.
    def formatMean (self,table):
        # Get a FeatureCollection with unique row IDs.
        rows = table.distinct(self.polygonID)
        filterEq = ee.Filter.equals(leftField=self.polygonID, rightField=self.polygonID)
        innerJoin = ee.Join.saveAll('matches')
        toyJoin = innerJoin.apply(primary=rows, secondary=table, condition=filterEq)
        
        return toyJoin.map(algorithm = self.processMatchesMean)
    
    ## method that enables and disables exportation of Std values
    #  @param[in] Bool is a boolean value True for enable and False for disable
    def exportStd(self,Bool):
        if (isinstance(Bool, bool)):
            self.exportStdVal = Bool
        else : 
            print ("WARNDING: function exportStd() takes as input a Boolean value!")
            
    ## @brief method that enables and disables exportation of Mean Values
    #  @param[in] Bool is a boolean value True for enable and False for disable
    def exportMean(self,Bool):
        if (isinstance(Bool, bool)):
            self.exportMeanVal = Bool
        else : 
            print ("WARNDING: function exportStd() takes as input a Boolean value!")
    
    
    # Fielddata: 
    def processMatchesStd(self,row):
        # Get the list of all features with a unique row ID.
        matches = ee.List(row.get('matches'))
        # Map a function over the list of rows to return a list of column ID and value.
        values = matches.map(lambda feature: [ee.Feature(feature).get('bandName'),
                                              ee.Feature(feature).get('stdDev')])
             
        # Return the row with its ID property and properties for all matching column IDs storing the output of the reducer.
        return row.select([self.polygonID]).set(ee.Dictionary(values.flatten()))


    ## Fielddata: Format a table of triplets into a 2D table of rowId x colId.
    def formatStd (self,table):
        # Get a FeatureCollection with unique row IDs.
        rows = table.distinct(self.polygonID)
        filterEq = ee.Filter.equals(leftField=self.polygonID, rightField=self.polygonID)
        innerJoin = ee.Join.saveAll('matches')
        toyJoin = innerJoin.apply(primary=rows, secondary=table, condition=filterEq)
        return toyJoin.map(algorithm = self.processMatchesStd)    
    
    # collection = s2bands
    def exportFeaturesMeanStdCSV(self,collection,ouutCsvFeatureVectors,driveFolder):
        if (self.currentSubset == None):
            raise Exception("Please call check _init_ since shp has not been loaded" )
        featureCollection = self.getFeatureCollection(collection)

        tableMean = self.formatMean(featureCollection)
        tableStd  = self.formatStd (featureCollection)
               
        meanName = ouutCsvFeatureVectors+"_mean"
        stdName = ouutCsvFeatureVectors+"_stdD"
        print ("START EXPORTING FEATURES VECTORS OF A SINGLE FILE")
        if (self.exportMeanVal):
            task = ee.batch.Export.table.toDrive(**{
                'collection':tableMean,
                'description':meanName,
                'folder': driveFolder,
                'fileFormat':'CSV'
            })
            task.start()

        if (self.exportStdVal):
            task = ee.batch.Export.table.toDrive(**{
                'collection':tableStd,
                'description':stdName,
                'folder': driveFolder,
                'fileFormat':'CSV'
            })
            task.start()
    